In [1]:
import cv2
import numpy as np
import pandas as pd

# Vita renk skalası dosyasını yükleyin
csv = pd.read_csv('detailed_vita.csv')

# En yakın rengi bulmak için kullanılan fonksiyon
def getColorName(R, G, B):
    minimum = 10000
    cname = "Unknown"
    for i in range(len(csv)):
        d = abs(R - int(csv.loc[i, "R"])) + abs(G - int(csv.loc[i, "G"])) + abs(B - int(csv.loc[i, "B"]))
        if d <= minimum:
            minimum = d
            cname = csv.loc[i, "color_name"]
    return cname

# Resmi yükleyin
img_path = "images.jpeg"
img = cv2.imread(img_path)
img_resized = cv2.resize(img, (800, 400))  # Görüntüyü yeniden boyutlandırın

# Gri tonlamalı resme dönüştürün
gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

# Görüntüyü bulanıklaştırarak gürültüyü azaltın
blurred = cv2.medianBlur(gray, 5)

# Hough Daire Dönüşümü ile daireleri tespit edin
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
                           param1=50, param2=30, minRadius=10, maxRadius=50)

# Eğer daireler tespit edildiyse
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        # Dairenin içindeki ortalama rengi alın
        mask = np.zeros_like(img_resized)
        cv2.circle(mask, (x, y), r, (255, 255, 255), -1)
        mean_val = cv2.mean(img_resized, mask=mask[:,:,0])
        
        # Ortalama RGB değerini al
        r_val, g_val, b_val = int(mean_val[2]), int(mean_val[1]), int(mean_val[0])
        
        # En yakın Vita rengini bulun
        color_name = getColorName(r_val, g_val, b_val)
        
        # Dairenin etrafına çember çizin ve renk ismini yazdırın
        cv2.circle(img_resized, (x, y), r, (0, 255, 0), 3)
        cv2.putText(img_resized, color_name, (x - r, y - r - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

# Sonuçları gösterin
cv2.imshow("Detected Teeth Colors", img_resized)
cv2.waitKey(0)
cv2.destroyAllWindows()
